In [ ]:
# Copyright 2020 IITK EE604A Image Processing. All Rights Reserved.
# 
# Licensed under the MIT License. Use and/or modification of this code outside of EE604 must reference:
#
# © IITK EE604A Image Processing 
# https://github.com/ee604/ee604_assignments
#
# Author: Shashi Kant Gupta, Chiranjeev Prachand and Prof K. S. Venkatesh, Department of Electrical Engineering, IIT Kanpur

# Task 4 (Bonus Question): Template Matching and Visual Search

In this bonus task, you will study about template matching algorithm, which is used to search and find a template image in a larger image. You will then use this method and techniques you learned in task1, task2, and task3 to search for a given "target" image in a large cluttered "search" image. This task of finding a target image is usually called a visual search task.

In this assignment, you will be using this method in a preliminary setting. But this method can be easily generalized to complex visual search tasks. Instead of directly using the image pixels, you first extract features of those images using some deep learning methods or other computer vision algorithms and then apply the template matching method on extracted features instead of the image pixels.

**Theories:** 
* Template Matching: [http://www.cse.psu.edu/~rtc12/CSE486/lecture07.pdf](http://www.cse.psu.edu/~rtc12/CSE486/lecture07.pdf)
* Tempalte Matching (Optional): [https://docs.adaptive-vision.com/4.7/studio/machine_vision_guide/TemplateMatching.html](https://docs.adaptive-vision.com/4.7/studio/machine_vision_guide/TemplateMatching.html)
* Template Matching Wikipedia: [https://en.wikipedia.org/wiki/Template_matching](https://en.wikipedia.org/wiki/Template_matching)
* Visual Search (Optional): [http://www.scholarpedia.org/article/Visual_search](http://www.scholarpedia.org/article/Visual_search)

---

### Your Task
You will be given a set of visual search tasks containing 156 various target images to search in 156 corresponding search images. All image contains only English alphabets. You have to write a python function that will return the (x, y) position of the target in the given search image. You have to use the template matching approach to find the location of the target image. But there's a problem, somehow the provided dataset got corrupted, and it contains lots of noises in it. Smartly using some image enhancement methods to make the images clearer and then applying template matching algorithms could help. It's up to you how you use the techniques taught in EE604 lectures to get the best score. Your score will be evaluated as `score = num_correct_prediction/156`

* Assume the target image is of the same size in the search image.
* Assume the target image is in the same orientation in the search image.
* You are free to use any OpenCV module except `cv2.matchTemplate()`.

**Example of Visual Search Task:**
![](https://github.com/ee604/ee604_assignments/raw/master/assignment_2/imgs/Example.png)

---

In [45]:
%%bash
pip install git+https://github.com/ee604/ee604_plugins

  Cloning https://github.com/ee604/ee604_plugins to /tmp/pip-req-build-0luwgrmk
  Created wheel for ee604-plugins: filename=ee604_plugins-0.2.2-cp36-none-any.whl size=2313 sha256=09477071dc13a0f48931ca754b827559384cf30a46deef443b830dfada0fef8c
  Stored in directory: /tmp/pip-ephem-wheel-cache-jxus4rju/wheels/34/a8/1d/ae3b7d209ecde89b4800a47ec55a61e7503bb9548bbb975806
Successfully built ee604-plugins


  Running command git clone -q https://github.com/ee604/ee604_plugins /tmp/pip-req-build-0luwgrmk


In [46]:
# Importing required libraries

import cv2
import numpy as np
from IPython.display import display
from PIL import Image
import matplotlib.pyplot as plt

from ee604_plugins import download_dataset

download_dataset(assignment_no=2, task_no=4)

Download Complete!


In [47]:
X_img = np.load("data/search_img.npy")
Y_img = np.load("data/target_img.npy")
M_img = np.load("data/mask_img.npy")

sample_target = np.copy(Y_img[0])
sample_search = np.copy(X_img[0])

In [79]:
# Do not change codes inside this cell

def calc_search_score():
    c = 0 
    for k in range(len(X_img)):
        x, y = searchTarget(np.copy(X_img[k]), np.copy(Y_img[k]))
        c += M_img[k][x, y]
        #print(c)
    return round(c/len(X_img), 2)


def Normalised_Cross_Correlation(window, target):
    window = window - np.mean(window)
    target = target - np.mean(target)
    
    corr = np.sum(window * target)
    norm_factor = np.sqrt(np.sum(pow(window,2))) * np.sqrt(np.sum(pow(target,2)))

    return corr/norm_factor


def template_matching(img, target):
    m, n = img.shape
    tm, tn = target.shape
    
    (max_corr_i, max_corr_j) = (0, 0)
    Maximum = 0
    corr_matrix = np.zeros((m-tm, n-tn))

    for i in range(0,m-tm):
      for j in range(0,n-tn):
        window = img[i:i+tm, j:j+tn]
        corr_matrix[i][j] = Normalised_Cross_Correlation(window,target)

        if(corr_matrix[i][j]>Maximum):
          Maximum = corr_matrix[i][j]
          (max_corr_i,max_corr_j) = (i,j)

    return (max_corr_j, max_corr_i)


In [80]:
def searchTarget(search_img, target_img):
    '''    
    Inputs:                      
    + search_img - image in which you have to search for the target, size = (512, 512) ==> (dim1, dim2)
    + target_img - target image, size = (55, 55)
    
    Ouputs:
    + x - index along the first dimension ('dim1') of the search image where the target is present
    + y - index along the second dimension ('dim2') of the search image where the target is present
    
    Allowed external package:
    + You are free to use any OpenCV/numpy module except 'cv2.matchTemplate()'.
    
    Hint:
    + Make sure you properly map the index with respect to input image size i.e. (512, 512). 
      Generally convolution/ correlation operation will reduce your output size by the size 
      of templated image.
    + Correlation operation can be written in terms of convolution operator.
    + Use OpenCV's cv2.matchTemplate() to know which method works best and implement corresponding
      method using other openCV modules.
    + This algorithm can be implemented using the functions you wrote/ used for other three tasks.
    
    '''
    x, y = 0, 0
    
    #############################
    # Start your code from here #
    #############################
    
    less_noisy_search_img = cv2.GaussianBlur(search_img, (7,7), 1)
    less_noisy_target_img = cv2.GaussianBlur(target_img, (5,5), 1)
    '''
    v = cv2.matchTemplate(less_noisy_search_img,less_noisy_target_img,method = cv2.TM_CCOEFF_NORMED)
    a,b,c,d = cv2.minMaxLoc(v)
    y,x = d
    '''
    y,x = template_matching(less_noisy_search_img, less_noisy_target_img)
    
    #############################
    # End your code here ########
    ############################# 
    
    return x, y

In [81]:
# Do not change codes inside this cell
score = calc_search_score()
print("Your score:", score)

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
12.0
13.0
13.0
14.0
15.0
16.0
16.0
17.0
18.0
19.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
44.0
45.0
46.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
Your score: 0.94


In [ ]:
#It takes upto about 30 mins to run. It gives the ans as 0.94. It matches 146/156 pairs of (source, target) image.